# lstm_nn_predict.ipynb

## Notebook Purpose
This notebook is designed to use the trained LSTM model to predict future cryptocurrency prices. It will load the trained LSTM model, apply it to the test data, and generate future predictions. The notebook will also visualize the predictions and compare them with the actual values.

## Instructions
1. **Import Necessary Libraries**:
   - Import `pandas` for data manipulation.
   - Import `joblib` to load the trained model.
   - Import `matplotlib` for data visualization.

2. **Load Preprocessed Data and Model**:
   - Load the preprocessed CSV file created in the data preparation notebook.
   - Load the trained LSTM model from the model generation notebook.

3. **Generate Predictions**:
   - Use the LSTM model to predict future prices.
   - Apply the model to the test data to predict prices.

4. **Visualize Predictions**:
   - Generate a plot to compare the predicted prices with the actual closing prices.

5. **Save Predictions**:
   - Save the generated predictions to a new CSV file for future reference.

## Example Code
```python
# Import necessary libraries
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Load preprocessed data and model
data_path = 'data/cleaned_data/BTC_cleaned.csv'  # Update this path based on the selected cryptocurrency
data = pd.read_csv(data_path, parse_dates=['Date'], index_col='Date')
model_path = 'models/trained_lstm_model.h5'
model = load_model(model_path)

# Prepare data for prediction
X_test = data['Close'].values.reshape(-1, 1)

# Make predictions
predictions = model.predict(X_test)

# Inverse transform predictions if necessary
predictions = scaler.inverse_transform(predictions)

# Save predictions
results = data[['Close']].copy()
results['Predictions'] = predictions
results.to_csv('results/lstm_predictions.csv')

# Display predictions
results.head()

# Plot predictions vs actual prices
plt.figure(figsize=(14, 7))
plt.plot(results.index, results['Close'], label='Actual Price')
plt.plot(results.index, results['Predictions'], label='Predicted Price')
plt.title('LSTM Model Predictions vs Actual Prices')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model


In [ ]:
# Load preprocessed data
data_path = 'data/cleaned_data/BTC_cleaned.csv'  # Update this path based on the selected cryptocurrency
prices_df = pd.read_csv(data_path, parse_dates=['time'], index_col='time')


In [ ]:
# Load the trained LSTM model
model_path = 'models/trained_lstm_model.h5'
nn_model = load_model(model_path)


In [ ]:
# Function to create sequences of data
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length), 0]
        y = data[i+seq_length, 0]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)


In [ ]:
# Extract closing prices
close = prices_df['close'].values.reshape(-1,1)
scaler = MinMaxScaler(feature_range=(0,1))
scaled_close = scaler.fit_transform(close)

# Create sequences based on the number of days
seq_length = 60
X, y = create_sequences(scaled_close, seq_length)

# Split into training and testing data
split = int(len(X) * .80)
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

# Reshape data for LSTM input [samples, time steps, features]
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


In [ ]:
# Generate predictions using the trained LSTM model
y_pred = nn_model.predict(X_test)

# Inverse transform the predictions and actual values to get actual prices
y_pred = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))


In [ ]:
# Compare predictions vs actual values
plt.figure(figsize=(14, 7))
plt.plot(prices_df.index[-len(y_test):], y_test_inv, label='Actual Prices')
plt.plot(prices_df.index[-len(y_test):], y_pred, label='Predicted Prices', linestyle='--')
plt.title('Actual vs Predicted Prices')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()


In [ ]:
# Predict future prices
X_future = []
start_index = len(scaled_close) - seq_length
for i in range(start_index, start_index + 30):
    seq = scaled_close[i-seq_length:i,0]
    X_future.append(seq)

# Convert X_future to numpy array and reshape for LSTM input [samples, time steps, features]
X_future = np.array(X_future)
X_future = np.reshape(X_future, (X_future.shape[0], X_future.shape[1], 1))

# Predict future prices
nn_future_predictions = nn_model.predict(X_future)
nn_future_predictions = scaler.inverse_transform(nn_future_predictions)


In [ ]:
# Plot future predictions
last_date = prices_df.index[-1]
next_30_days = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=30)

plt.figure(figsize=(14, 7))
plt.plot(prices_df['close'], label='Historical Prices')
plt.plot(next_30_days, nn_future_predictions, label='Future Predictions', linestyle='--')
plt.title('Future Price Predictions using LSTM Neural Network')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()

# Save the future predictions
future_predictions_df = pd.DataFrame({
    'Date': next_30_days,
    'Predicted_Price': nn_future_predictions.flatten()
})
future_predictions_df.to_csv('results/output_predictions/future_predictions.csv', index=False)
